In [1]:
!pip install gradio

     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.0 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 211 kB 50.6 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 253 kB 39.3 MB/s 
     |████████████████████████████████| 2.0 MB 34.5 MB/s 
     |████████████████████████████████| 144 kB 36.9 MB/s 
     |████████████████████████████████| 271 kB 13.6 MB/s 
     |████████████████████████████████| 94 kB 1.5 MB/s 
     |████████████████████████████████| 10.9 MB 21.5 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
     |████████████████████████████████| 79 kB 6.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 61 kB 465 kB/s 
     |████████████████████████████████| 856 kB 35.5 MB/s 
     |████████████████████████

In [4]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-3j4xofho
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-3j4xofho
     |████████████████████████████████| 50 kB 2.6 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
     |████████████████████████████████| 147 kB 9.8 MB/s 
     |████████████████████████████████| 130 kB 36.6 MB/s 
     |████████████████████████████████| 38.1 MB 10.4 MB/s 
     |████████████████████████████████| 749 kB 38.3 MB/s 
     |████████████████████████████████| 843 kB 37.0 MB/s 
     |████████████████████████████████| 596 kB 33.0 MB/s 
     |████████████████████████████████| 112 kB 38.8 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp37-cp37m-linux_x86_64.whl size=5697962 sha256=0a85463fcfd9f053cbdb888d283ebca3b457700967ccc48248ef1bf070b1761d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ksr9pr6f/wheels/07/dc/32/0322cb484dbefab8b9366bfedbaff5060ac7

In [37]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 


In [8]:
import gradio as gr
import numpy as np
import pandas as pd

import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import plotly.express as px

In [6]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [38]:
import transformers

## Example with different plots

In [3]:
plot_types = ["Matplotlib", "Plotly"]
months = ["January", "February", "March", "April", "May"]
countries = ["USA", "Canada", "Mexico", "UK"]

def outbreak(plot_type, r, month, countries_, social_distancing):
    m = months.index(month)
    start_day = 30 * m
    final_day = 30 * (m + 1)
    x = np.arange(start_day, final_day + 1)
    pop_count = {"USA": 350, "Canada": 40, "Mexico": 300, "UK": 120}
    if social_distancing:
        r = np.sqrt(r)
    df = pd.DataFrame({'day': x})
    for country in countries_:
        df[country] = ( x ** (r) * (pop_count[country] + 1))
        

    if plot_type == "Matplotlib":
        fig = plt.figure()
        plt.plot(df['day'], df[countries_])
        plt.title("Outbreak in " + month)
        plt.ylabel("Cases")
        plt.xlabel("Days since Day 0")
        plt.legend(countries_)
        return fig
    elif plot_type == "Plotly":
        fig = px.line(df, x='day', y=countries_)
        fig.update_layout(title="Outbreak in " + month,
                   xaxis_title="Cases",
                   yaxis_title="Days Since Day 0")
        return fig
    else:
        raise NotImplementedError("Only Matplotlib and Plotly plot types are aviliable.")



iface = gr.Interface(
    outbreak,
    [
        gr.inputs.Dropdown(plot_types, label="Plot Type"),
        gr.inputs.Slider(1, 4, default=3.2, label="R"),
        gr.inputs.Dropdown(months, label="Month"),
        gr.inputs.CheckboxGroup(countries, label="Countries", default=["USA", "Canada"]),
        gr.inputs.Checkbox(label="Social Distancing?"),
    ],
    gr.outputs.Plot(type="auto"),
)

iface.launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://58227.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f721d758e50>,
 'http://127.0.0.1:7860/',
 'https://58227.gradio.app')

## Example with image as ouput

In [22]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)

model_final_cafdb1.pkl: 261MB [00:14, 18.3MB/s]                           


In [33]:
def intance_segmentation(input_img):
    input_img_cv2 = cv2.merge([input_img[..., 2], input_img[..., 1], input_img[..., 0]])
    input_img_cv2 = cv2.resize(input_img_cv2, (input_img.shape[1] // 4, input_img.shape[0] // 4))
    outputs = predictor(input_img_cv2)
    visualizer = Visualizer(
        input_img_cv2[:, :, ::-1], 
        MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), 
        scale=1.2
    )
    panoptic_seg, segments_info = outputs["panoptic_seg"]
    visualizer = visualizer.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
    return visualizer.get_image()

In [34]:
iface = gr.Interface(
    intance_segmentation, 
    gr.inputs.Image(), 
    "image"
)
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://44493.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f721d758e50>,
 'http://127.0.0.1:7862/',
 'https://44493.gradio.app')

## Text generation example

In [39]:
title = "Ask Rick a Question"
description = """
<center>
The bot was trained to answer questions based on Rick and Morty dialogues. Ask Rick anything!
<img src="https://huggingface.co/spaces/course-demos/Rick_and_Morty_QA/resolve/main/rick.png" width=200px>
</center>
"""

article = "Check out [the original Rick and Morty Bot](https://huggingface.co/spaces/kingabzpro/Rick_and_Morty_Bot) that this demo is based off of."

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("ericzhou/DialoGPT-Medium-Rick_v2")
model = AutoModelForCausalLM.from_pretrained("ericzhou/DialoGPT-Medium-Rick_v2")

def predict(input):
    # tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # generate a response 
    history = model.generate(new_user_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()

    # convert the tokens to text, and then split the responses into the right format
    response = tokenizer.decode(history[0]).split("<|endoftext|>")
    return response[1]

gr.Interface(fn = predict, inputs = ["textbox"], outputs = ["text"], title = title, description = description, article = article).launch() 


Downloading:   0%|          | 0.00/617 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/893 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://45978.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f721d758e50>,
 'http://127.0.0.1:7862/',
 'https://45978.gradio.app')